<center><h1> IFT-6758  Data Science  </h1></center>
<center><h2> Fall - 2021 </h2></center> 
<center><h3> Team : Kevin Traoré, Etienne Delisle, Paul Villedieu, Mohammed Laredj </h3></center> 
<center><h3> Interactive Debugging Tool </h3></center> 



In [1]:
jsonDataFolder = '../jsondata'
datasetsFolder = '../datasets'

In [2]:
import ipywidgets as widgets
from IPython.display import display
from typing import List
import time
import os
import json
from ift6758.data.question_2 import nhl_data
from ift6758.data import seasonTypes, seasonCodes, get_season_type_label, get_season_years_label,\
Event, Game, Team, EventBuilder, GameBuilder, TeamBuilder, create_folder_if_not_exists, get_opponent_goal_coordinates

create_folder_if_not_exists(jsonDataFolder)
create_folder_if_not_exists(datasetsFolder)

print('ipywidgets version :', widgets.__version__)

ipywidgets version : 7.6.5


In [3]:
from datetime import datetime
firstRecordedSeason = 1917
nextSeasonYear = datetime.now().year
nbGameFrom2017 = 1271
nbGameBefore2017 = 1230

def get_seasons_list():
    result = []
    seasonYear = firstRecordedSeason
    while seasonYear < nextSeasonYear:
        result.append(str(seasonYear) + ' - ' + str(seasonYear + 1))
        seasonYear += 1
    
    return result


In [4]:
def get_game_details_label(game:Game):
    return f"""{game.homeTeam.name} ( {game.homeTeam.triCode} )    vs    {game.awayTeam.name} ({game.awayTeam.triCode})
Venue : {game.venueName}
Started at : {game.startDatetime}
Ended at : {game.endDatetime}"""

def get_teams_details_label(game:Game):
    return f"""                  Home    Away
Teams :           {game.homeTeam.triCode}     {game.awayTeam.triCode}
Goals :           {game.homeTeam.goals}       {game.awayTeam.goals}
SoG :             {game.homeTeam.shotsOnGoal}      {game.awayTeam.shotsOnGoal}
SO Goals :        {game.homeTeam.shootoutScores}       {game.awayTeam.shootoutScores}
SO Attempts :     {game.homeTeam.shootoutAttempts}       {game.awayTeam.shootoutAttempts}"""

def get_event_details_label(anEvent):
    return f"""         {anEvent.description}
         {anEvent.periodTime} P-{anEvent.periodNumber}
         {anEvent.periodRemainingTime} remaining of {anEvent.periodType.lower()} period
"""


In [5]:
def get_game_json_file_name(gameNum):
    return f'{jsonDataFolder}/{gameNum}.json'

def load_game(gameNum):
    with open(get_game_json_file_name(gameNum), 'r') as file:
        game_data = file.read()

    return GameBuilder().build(json.loads(game_data))


def get_season_games(seasonStartYear, gameTypeCode):
    result = nhl_data([int(seasonStartYear)], [gameTypeCode], jsonDataFolder, light=True)

    for gameId in result:
        if not os.path.exists(get_game_json_file_name(gameId)):
            result.remove(gameId)

    return result


def get_season_type_code(selectedSeasonLabel):
    return seasonCodes[selectedSeasonLabel]

In [6]:
def axes_points_formatter(x, pos):
    import math
    frac, whole = math.modf(x)

    if abs(frac) in [0.25, 0.75]:
        return f'{x:.2f}'
    
    return f'{x:.1f}'


def get_title(name_left_team, name_right_team):
    title_max_length = 76
    
    spaces_length = (title_max_length - len(name_left_team) - len(name_right_team)) // 2
    
    return name_left_team + (' ' * spaces_length) + name_right_team


def plot_event(coordinates, name_left_team, name_right_team):    
    import matplotlib.pyplot as plt
    import numpy as np
    data = plt.imread('../figures/nhl_rink.png')

    xaxis_min = -100
    xaxis_max = 100
    yaxis_min = -42.5
    yaxis_max = 42.5

    plt.imshow(data, extent=[xaxis_min, xaxis_max, yaxis_min, yaxis_max])

    axes = plt.gca()
    axes.set_facecolor('m')
    axes.set_title(get_title(name_left_team, name_right_team))

    xaxis_step = 25.0
    yaxis_step = 21.25
    axes.set_xticks(np.arange(xaxis_min, xaxis_max + 1, xaxis_step))
    axes.set_yticks(np.arange(yaxis_min, yaxis_max + 1, yaxis_step))

    axes.xaxis.set_major_formatter(axes_points_formatter)
    axes.yaxis.set_major_formatter(axes_points_formatter)

    feet_label = 'feet'
    axes.set_xlabel(feet_label)
    axes.set_ylabel(feet_label)

    if len(coordinates) > 1:
        axes.scatter([coordinates['x']], [coordinates['y']], marker='8', color='b', s=[150])

    plt.show()


In [7]:
def get_left_and_right_team(anEvent, homeTeamTriCode, awayTeamTriCode, periodsDict):
    eventCoordinates = anEvent.coordinates
    if eventCoordinates == None or len(eventCoordinates) < 2:
        return '', ''

    awayTeamGoal = get_opponent_goal_coordinates(homeTeamTriCode, homeTeamTriCode,\
                                                 eventCoordinates, anEvent.periodNumber, periodsDict)
    if awayTeamGoal == None:
        return '', ''
    elif awayTeamGoal['x'] < 0:
        return awayTeamTriCode, homeTeamTriCode
    else:
        return homeTeamTriCode, awayTeamTriCode

In [8]:
seasonsList = get_seasons_list()
seasonsMaxIndex = len(seasonsList) - 1
seasonTypesList = list(seasonTypes.values())
seasonsDropdownDefaultValue = seasonsList[seasonsMaxIndex]
seasonTypeRadioButtonsDefaultValue = seasonTypesList[0]

playInterval = 3000

gamesList = get_season_games(seasonsDropdownDefaultValue[:4],\
                             get_season_type_code(seasonTypeRadioButtonsDefaultValue))
gamesMaxIndex = len(gamesList) - 1
currentGameIndex = 0
currentGame = load_game(gamesList[currentGameIndex])

eventsList = currentGame.events
eventsMaxIndex = len(eventsList) - 1
currentEventIndex = 0
currentEvent = eventsList[currentEventIndex]

seasonsDropdown = widgets.Dropdown(
    options=seasonsList,
    value=seasonsDropdownDefaultValue,
    layout={'width': '200px'},
    description='Season : ',
    disabled=False,
)

seasonTypeRadioButtons = widgets.RadioButtons(
    options=seasonTypesList,
    value=seasonTypeRadioButtonsDefaultValue,
    description='Type :',
    disabled=False
)

seasonSelectionBox = widgets.HBox(
    [
        seasonsDropdown,
        seasonTypeRadioButtons
    ]
)

teamsDetailsLayout = widgets.Layout()
teamsDetailsLayout.margin = '0 0 0 150px'

gameDetailsOutput = widgets.Output()
teamsDetailsOutput = widgets.Output(layout=teamsDetailsLayout)
eventsDetailsOutput = widgets.Output(layout=teamsDetailsLayout)
outputToDelete = widgets.Output()

gamesSlider = widgets.IntSlider(max=gamesMaxIndex)
eventsPlayer = widgets.Play(
    value=currentGameIndex,
    min=0,
    max=gamesMaxIndex,
    step=1,
    interval=playInterval,
    description="Play all games",
    disabled=False
)
eventsSlider = widgets.IntSlider(max=eventsMaxIndex)
eventsPlayerBox = widgets.HBox(
    [
        eventsPlayer,
        eventsSlider
    ],
    layout=teamsDetailsLayout
)

def update_game_details(aGame):
    global gameDetailsOutput, teamsDetailsOutput
    
    with gameDetailsOutput:
        gameDetailsOutput.clear_output()
        print(get_game_details_label(aGame))
    
    with teamsDetailsOutput:
        teamsDetailsOutput.clear_output()
        print(get_teams_details_label(aGame))

def on_game_change(newGameInfo):
    global currentGameIndex, gamesList, currentGame, eventsList, eventsMaxIndex,\
            currentEventIndex, eventsSlider, currentEvent
    
    currentGameIndex = newGameInfo['new']
    currentGame = load_game(gamesList[currentGameIndex])
    update_game_details(currentGame)
    eventsList = currentGame.events
    eventsMaxIndex = len(eventsList) - 1
    currentEventIndex = 0
    eventsSlider.value = currentEventIndex
    currentEvent = eventsList[currentEventIndex]
    update_event_details(currentEvent)

def update_event_details(anEvent):
    global eventsDetailsOutput, currentGame
    
    with eventsDetailsOutput:
        eventsDetailsOutput.clear_output()
        print(get_event_details_label(anEvent))
        leftTeam, rightTeam = get_left_and_right_team(anEvent, currentGame.homeTeam.triCode,\
                                                      currentGame.awayTeam.triCode, currentGame.periods)
        plot_event(anEvent.coordinates, leftTeam, rightTeam)

def on_event_change(newEventInfo):
    global currentEventIndex, currentEvent, gamesSlider, eventsList, eventsMaxIndex,\
            gamesSlider, currentGameIndex, gamesMaxIndex, outputToDelete
    
    newEventIndex = newEventInfo['new']
    currentEventIndex = newEventIndex
    currentEvent = eventsList[currentEventIndex]
    update_event_details(currentEvent)

def on_season_change(change):
    global gamesList, seasonsDropdown, seasonTypeRadioButtons, gamesMaxIndex, gamesSlider
    
    print(seasonsDropdown.value)
    print(seasonTypeRadioButtons.value)
    gamesList = get_season_games(seasonsDropdown.value[:4],\
                             get_season_type_code(seasonTypeRadioButtons.value))
    gamesMaxIndex = len(gamesList) - 1
    gamesSlider.value = 0
    on_game_change({'new' : 0})

gamesSlider.observe(on_game_change, names='value')
widgets.jslink((eventsPlayer, 'value'), (eventsSlider, 'value'))
eventsSlider.observe(on_event_change, names='value')
seasonsDropdown.observe(on_season_change, names='value')
seasonTypeRadioButtons.observe(on_season_change, names='value')

gamesSlider.value=currentGameIndex
eventsSlider.value=currentEventIndex

display(seasonSelectionBox, gamesSlider, gameDetailsOutput,
        teamsDetailsOutput, eventsPlayerBox, eventsDetailsOutput, outputToDelete)

update_game_details(currentGame)
update_event_details(currentEvent)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1271/1271 [01:03<00:00, 20.04it/s]


IntSlider(value=0, max=867)

Output()

Output(layout=Layout(margin='0 0 0 150px'))

Output(layout=Layout(margin='0 0 0 150px'))

Output()